In [7]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import desc
from pyspark.sql.functions import countDistinct

In [2]:
spark_avia = SparkSession.builder.master("yarn").appName("avia").config("spark.jars", "/home/teran45/spark/postgresql-42.3.4.jar").getOrCreate()

2022-05-26 01:19:16,801 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-26 01:19:18,609 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
df = spark_avia.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/dataengineering") \
    .option("dbtable", "public.opensky") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [14]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- icao24: string (nullable = true)
 |-- callsign: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- time_position: integer (nullable = true)
 |-- last_contact: integer (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- baro_altitude: double (nullable = true)
 |-- on_ground: boolean (nullable = true)
 |-- velocity: double (nullable = true)
 |-- true_track: double (nullable = true)
 |-- vertical_rate: double (nullable = true)
 |-- sensors: integer (nullable = true)
 |-- geo_altitude: double (nullable = true)
 |-- squawk: string (nullable = true)
 |-- spi: boolean (nullable = true)
 |-- position_source: integer (nullable = true)



In [16]:
df.createOrReplaceTempView("EMP")

In [28]:
avia = spark_avia.sql('select origin_country, count( distinct icao24) as avia_amount from EMP group by origin_country order by avia_amount DESC')

In [31]:
avia.coalesce(1).write.csv("hdfs:///user/teran45/data/avia/country")

In [30]:
avia.show()

+--------------------+-----------+
|      origin_country|avia_amount|
+--------------------+-----------+
|       United States|      15460|
|           Australia|       1292|
|              Canada|       1094|
|             Germany|       1073|
|      United Kingdom|        942|
|              France|        579|
|               India|        568|
|              Turkey|        530|
|               Japan|        510|
|               China|        491|
|               Spain|        473|
|             Ireland|        462|
|  Russian Federation|        462|
|United Arab Emirates|        461|
|              Brazil|        403|
|Kingdom of the Ne...|        373|
|             Austria|        320|
|         Switzerland|        304|
|         New Zealand|        294|
|               Malta|        291|
+--------------------+-----------+
only showing top 20 rows

